In [1]:
import ot
import torch
from multiprocessing import Pool

#GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [13]:
def wasserstein_distance(a, b, wa, wb):
        
    W_dist = ot.wasserstein_1d(a, b, wa, wb, p = 1).to(device)

    return W_dist

In [3]:
def fun(a, b, wa, wb):
    W_dist = torch.zeros(wa.size(0))

    for i in range(wa.size(0)):
        W_dist[i] = wasserstein_distance(a, b, wa[i], wb[i])

    return W_dist

In [4]:
def dataset_maker(n, m):
    
    zero = torch.zeros(m, n).to(device)
    one = torch.zeros(m, n).to(device)
    two = torch.zeros(m, n).to(device)
    three = torch.zeros(m, n).to(device)

    for i in range(m):
        zero[i] =  torch.tensor(ot.datasets.make_1D_gauss(n, m = i + 10, s = 5))
        one[i] =  torch.tensor(ot.datasets.make_1D_gauss(n, m = i + 40, s = 5)) 
        two[i] =  torch.tensor(ot.datasets.make_1D_gauss(n, m = i + 50, s = 5))
        three[i] =  torch.tensor(ot.datasets.make_1D_gauss(n, m = i + 70, s = 5)) 

    return torch.stack([zero, one, two, three])


In [5]:
def iterable(data, x):

    for i in range(data.size(0)):

        yield([x, x, data[0], data[i]])

In [6]:
m = 100
n = 100

dataset = dataset_maker(m, n).to(device)

In [15]:
import time 

x = torch.arange(n).to(device)

W_dist = torch.zeros(dataset.size(0), dataset.size(1)).to(device)

st = time.time()
for i in range(dataset.size(0)):

    W_dist[i] = fun(x, x, dataset[0], dataset[i]).to(device)

et = time.time()

print(et-st)

0.46358418464660645
